## Prepare Callback

In [1]:
import os

In [2]:
%pwd

'c:\\Users\\harik\\Desktop\\All Folder\\PGDA\\Interships & Projects\\NLP\\End-to-End Workflow\\cnnClassifier\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\harik\\Desktop\\All Folder\\PGDA\\Interships & Projects\\NLP\\End-to-End Workflow\\cnnClassifier'

In [5]:
# Now we define the entity

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen= True)
class PrepareCallbackConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [7]:
# Now we define configuration manager

In [8]:
from cnnClassifier.constants import *
from cnnClassifier.utils import read_yaml, create_directories

In [9]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):

            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)

            create_directories([self.config.artifacts_root])


    def get_prepare_callback_config(self) -> PrepareCallbackConfig:
          
          config = self.config.prepare_callbacks
          model_ckpt_dir= os.path.dirname(config.checkpoint_model_filepath)   # Now it ignores the extension (model.h5) and only creating the dir

          create_directories([
                Path(model_ckpt_dir),
                Path(config.tensorboard_root_log_dir)  # Here we dont need to create root dir again as it will be automatically created  with these codes                                                        
          ])

          prepare_callback_config = PrepareCallbackConfig(               # Now we calling the entity we prepared before
                root_dir= Path(config.root_dir),
                tensorboard_root_log_dir= Path(config.tensorboard_root_log_dir),
                checkpoint_model_filepath= Path(config.checkpoint_model_filepath)
          )

          return prepare_callback_config

In [10]:
# Now we need to update components

In [11]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [12]:
class PrepareCallback:

    def __init__(self, config: PrepareCallbackConfig):
        self.config= config


    @property
    def _create_tb_callback(self):
        timestamp= time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir= os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}"
        )
        return tf.keras.callbacks.TensorBoard(log_dir= tb_running_log_dir)
    
    """ 
        When we use this property decorator, if we are calling this fuction (object) else where we dont actually need to specify the () to call
        it. We can access this object like its a attribute.

    """

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath= str(self.config.checkpoint_model_filepath),
            save_best_only= True
        )
    
    def get_tb_ckpt_callbacks(self):
        return[
            self._create_tb_callback,
            self._create_ckpt_callbacks
        ]

In [13]:
# Now we need to write the pipeline

In [14]:
try:
    config= ConfigurationManager()
    prepare_callback_config= config.get_prepare_callback_config()
    prepare_callbacks= PrepareCallback(config= prepare_callback_config)
    callback_list= prepare_callbacks.get_tb_ckpt_callbacks()

except Exception as e:
    raise e

[2023-10-05 16:25:01,623: INFO: common: yaml file: config\config.yaml loaded succesfully]
[2023-10-05 16:25:01,639: INFO: common: yaml file: params.yaml loaded succesfully]
[2023-10-05 16:25:01,639: INFO: common: created directory at: artifacts]
[2023-10-05 16:25:01,639: INFO: common: created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2023-10-05 16:25:01,648: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]
